In [ ]:
from google.colab import drive
import pandas as pd
import numpy as np
from statistics import mean
from itertools import cycle
from gensim.models import Word2Vec
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import recall_score
from sklearn.neighbors import NearestNeighbors

In [ ]:
pip install gensim

## Загрузка и знакомство с данными

In [ ]:
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!cp /content/gdrive/'My Drive'/ML .

cp: -r not specified; omitting directory '/content/gdrive/My Drive/ML'


In [ ]:
cols = [
    'sale_date_date', 'contact_id', 'shop_id', 'product_id',
    'product_sub_category_id', 'product_category_id', 'brand_id', 'quantity'
]
check_data = pd.read_csv('/content/gdrive/MyDrive/ML/Data_check.csv', usecols=cols)

<ipython-input-7-9b3bf04be5b2>:5: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  check_data = pd.read_csv('/content/gdrive/MyDrive/ML/Data_check.csv', usecols=cols)


In [ ]:
print(check_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000003 entries, 0 to 20000002
Data columns (total 8 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               float64
 4   product_sub_category_id  float64
 5   product_category_id      float64
 6   brand_id                 float64
 7   quantity                 object 
dtypes: float64(5), object(3)
memory usage: 1.2+ GB
None


In [ ]:
check_data['sale_date_date'].value_counts()

2018-11-16        168713
2018-11-15        167893
2018-11-21        167858
2018-11-20        164768
2018-12-07        164278
                   ...  
2018-07-02         11067
2018-06-22          1824
2018-06-26           811
2018-12-09           133
(затронуто стр         1
Name: sale_date_date, Length: 215, dtype: int64

In [ ]:
check_data['contact_id'].value_counts()

730404         13905
665607          8677
795400          8264
995207          7455
749095          7377
               ...  
2355885            1
925767             1
956510             1
2682072            1
к: 20000000        1
Name: contact_id, Length: 1646435, dtype: int64

In [ ]:
check_data['shop_id'].value_counts()

313.0     267255
323.0     259413
350.0     255904
316.0     251810
269.0     242599
           ...  
2081.0         1
719.0          1
1202.0         1
1688.0         1
1640.0         1
Name: shop_id, Length: 851, dtype: int64

In [ ]:
check_data['product_id'].value_counts()

129195.0    169191
168308.0    118106
158662.0     89851
110629.0     87792
137539.0     77153
             ...  
120494.0         1
224621.0         1
317844.0         1
234526.0         1
193550.0         1
Name: product_id, Length: 36549, dtype: int64

In [ ]:
check_data['product_sub_category_id'].value_counts()

 738.0    2555316
 397.0     841103
 595.0     749197
-1.0       652908
 404.0     588604
           ...   
 832.0          1
 886.0          1
 809.0          1
 483.0          1
 660.0          1
Name: product_sub_category_id, Length: 440, dtype: int64

In [ ]:
check_data['product_category_id'].value_counts()

170.0    2969444
162.0    1906617
135.0    1890878
101.0    1363400
146.0    1009485
          ...   
184.0          8
203.0          6
149.0          5
207.0          4
145.0          4
Name: product_category_id, Length: 112, dtype: int64

In [ ]:
check_data['brand_id'].value_counts()

-1.0       16176371
 1520.0      881070
 1348.0      151294
 2038.0      105401
 4538.0       97269
             ...   
 1433.0           1
 4877.0           1
 932.0            1
 1539.0           1
 4529.0           1
Name: brand_id, Length: 1631, dtype: int64

In [ ]:
check_data['quantity'].value_counts()

1,00      16729626
2,00       1870682
3,00        431410
5,00        211800
4,00        171686
            ...   
1,03             1
0,945            1
202,00           1
15,50            1
0,562            1
Name: quantity, Length: 1244, dtype: int64

## Проверка данных на предмет аномалий и их исправление

In [ ]:
print("\nПропущенные значения в чековых данных:")
print(check_data.isna().sum())


Пропущенные значения в чековых данных:
sale_date_date             2
contact_id                 2
shop_id                    3
product_id                 3
product_sub_category_id    3
product_category_id        3
brand_id                   3
quantity                   3
dtype: int64


In [ ]:
check_data.dropna(inplace=True)

In [ ]:
print("\nЧековые данные:")
print(check_data)


Чековые данные:
         sale_date_date contact_id  shop_id  product_id  \
0            2018-12-07    1260627   1455.0    168308.0   
1            2018-12-07     198287    279.0    134832.0   
2            2018-12-07    2418385    848.0    101384.0   
3            2018-12-07    1285774   1511.0    168570.0   
4            2018-12-07    1810323   1501.0    168319.0   
...                 ...        ...      ...         ...   
19999995     2018-06-13    1601618   1499.0     66842.0   
19999996     2018-06-13    1394104   1495.0    136795.0   
19999997     2018-06-13    1570654   1516.0    119513.0   
19999998     2018-06-13    1924036   1485.0     71723.0   
19999999     2018-06-13    1788779   1503.0     66164.0   

          product_sub_category_id  product_category_id  brand_id quantity  
0                           906.0                205.0      -1.0     1,00  
1                           404.0                 93.0      -1.0     1,00  
2                           404.0             

In [ ]:
check_data['product_id'] = check_data['product_id'].astype(str)

In [ ]:
check_data['quantity'] = check_data['quantity'].str.replace(',', '.').astype(float)

# Выводим обновленные данные
print("\nЧековые данные после исправлений:")
print(check_data)


Чековые данные после исправлений:
         sale_date_date contact_id  shop_id product_id  \
0            2018-12-07    1260627   1455.0   168308.0   
1            2018-12-07     198287    279.0   134832.0   
2            2018-12-07    2418385    848.0   101384.0   
3            2018-12-07    1285774   1511.0   168570.0   
4            2018-12-07    1810323   1501.0   168319.0   
...                 ...        ...      ...        ...   
19999995     2018-06-13    1601618   1499.0    66842.0   
19999996     2018-06-13    1394104   1495.0   136795.0   
19999997     2018-06-13    1570654   1516.0   119513.0   
19999998     2018-06-13    1924036   1485.0    71723.0   
19999999     2018-06-13    1788779   1503.0    66164.0   

          product_sub_category_id  product_category_id  brand_id  quantity  
0                           906.0                205.0      -1.0       1.0  
1                           404.0                 93.0      -1.0       1.0  
2                           404.0    

In [ ]:
check_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20000000 entries, 0 to 19999999
Data columns (total 8 columns):
 #   Column                   Dtype  
---  ------                   -----  
 0   sale_date_date           object 
 1   contact_id               object 
 2   shop_id                  float64
 3   product_id               object 
 4   product_sub_category_id  float64
 5   product_category_id      float64
 6   brand_id                 float64
 7   quantity                 float64
dtypes: float64(5), object(3)
memory usage: 1.3+ GB


## Рекомендации с Word2Vec

In [ ]:
# Группируем данные по дате, магазину и клиенту, а затем объединениняем 'product_id' в списки
grouped_df = check_data.groupby(['sale_date_date', 'shop_id', 'contact_id'])['product_id'].apply(list).reset_index()

In [ ]:
word2vec_model = Word2Vec(sentences=grouped_df['product_id'], vector_size=100, window=5, min_count=5, workers=4)

# Дополнительные параметры Word2Vec:
# - vector_size: размер вектора
# - window: максимальное расстояние между текущим и предсказываемым словом в предложении
# - min_count: игнорировать все слова, которые встречаются менее указанного числа раз

In [ ]:
def predict_recommendations(model, product_ids, top_n=3):
    """
    Предсказывает топ-3 рекомендаций для списка клиентов на основе эмбеддингов продуктов.

    Parameters:
    -----------
    model : gensim.models.word2vec.Word2Vec
        Обученная модель Word2Vec, содержащая эмбеддинги продуктов.

    product_ids : list of list of str
        Список списков идентификаторов продуктов для каждого клиента.

    top_n : int, optional (default=3)
        Количество рекомендаций, которое следует вернуть для каждого клиента.

    Returns:
    --------
    recommendations : list of list of str
        Список списков идентификаторов рекомендованных продуктов для каждого клиента.

    Examples:
    ---------
    >>> model = Word2VecModel(sentences=grouped_df['product_id'], vector_size=100, window=5, min_count=5, workers=4).model
    >>> product_ids = [['apple', 'banana'], ['orange', 'grape', 'banana']]
    >>> recommendations = predict_recommendations(model, product_ids, top_n=2)
    >>> print(recommendations)
    [['grape', 'orange'], ['apple', 'banana']]
    """
    recommendations = []
    for product_list in product_ids:
        product_embeddings = [model.wv[pid] for pid in product_list if pid in model.wv]

        if not product_embeddings:
            recommendations.append([])  # Если для данного клиента нет эмбеддингов продуктов
            continue

        # Усреднение эмбеддингов продуктов
        average_embedding = sum(product_embeddings) / len(product_embeddings)

        # Поиск наиболее похожих продуктов
        similar_products = model.wv.similar_by_vector(average_embedding, topn=top_n)

        # Извлечение только идентификаторов продуктов из результатов
        recommended_products = [pid for pid, _ in similar_products]
        recommendations.append(recommended_products)

    return recommendations

In [ ]:
# Пример: возьмем первые 5 записей для теста
test_data = grouped_df['product_id'][:5].tolist()
predicted_recommendations = predict_recommendations(word2vec_model, test_data)

for i, recommendations in enumerate(predicted_recommendations):
    print(f"Для клиента {grouped_df['contact_id'][i]} в магазине {grouped_df['shop_id'][i]} на дату {grouped_df['sale_date_date'][i]} рекомендуемые продукты: {recommendations}")

Для клиента 17453 в магазине 43.0 на дату 2018-01-01 рекомендуемые продукты: ['148022.0', '101264.0', '30656.0']
Для клиента 28331 в магазине 43.0 на дату 2018-01-01 рекомендуемые продукты: ['54592.0', '115174.0', '160776.0']
Для клиента 46184 в магазине 43.0 на дату 2018-01-01 рекомендуемые продукты: ['113585.0', '41890.0', '44789.0']
Для клиента 51940 в магазине 43.0 на дату 2018-01-01 рекомендуемые продукты: ['76308.0', '164043.0', '79058.0']
Для клиента 78417 в магазине 43.0 на дату 2018-01-01 рекомендуемые продукты: ['146135.0', '132911.0', '40243.0']


Оценка качества рекомендаций по метрике Precision@K:

In [ ]:
# Преобразование списков в последовательности
sequences = grouped_df['product_id'].tolist()

# Обучение модели Word2Vec
word2vec_model = Word2Vec(sentences=sequences, vector_size=100, window=5, min_count=5, workers=4)

# Получение эмбеддингов продуктов
product_embeddings = [word2vec_model.wv[str(product_id)] for sequence in sequences for product_id in sequence if str(product_id) in word2vec_model.wv]
product_ids = [int(float(pid)) for pid in grouped_df['product_id'] if str(pid).replace('.', '', 1).isdigit()]

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

def evaluate_recommendations(embeddings, product_ids, top_k=3):
    """
    Оценивает качество рекомендаций на основе встроенных представлений продуктов.

    Parameters:
    - embeddings (numpy.ndarray): Массив с встроенными представлениями продуктов.
    - product_ids (list): Список идентификаторов продуктов, для которых оценивается качество.
    - top_k (int): Количество ближайших соседей для оценки точности.

    Returns:
    - float: Значение precision@k, оценка качества рекомендаций.
    """

    # Создаем отображение индексов продуктов
    product_id_index_map = {pid: i for i, pid in enumerate(product_ids)}

    # Вычисляем матрицу сходства
    similarities = cosine_similarity(embeddings, embeddings)

    # Инициализируем счетчик точных предсказаний
    precision_at_k = 0.0

    # Итерируем по исходным продуктам
    for i, true_product_id in enumerate(product_ids):
        true_index = product_id_index_map[true_product_id]
        most_similar_indices = np.argsort(similarities[true_index])[::-1][1:top_k+1]
        precision_at_k += int(true_index in most_similar_indices)

    precision_at_k /= len(product_ids)
    return precision_at_k

precision_at_k = evaluate_recommendations(product_embeddings, product_ids, top_k=3)
print(f'Precision@3: {precision_at_k}')

## Генерируем пары фича-таргет

In [ ]:
def generate_batches(check_data, batch_size=32):
    """
    Генерирует батчи пар feature-target для заданного набора данных.

    Параметры:
    - check_data (pd.DataFrame): Входной DataFrame, содержащий столбцы 'product_id' и 'quantity'.
    - batch_size (int, optional): Размер каждого пакета. По умолчанию 32.

    Возвращает:
    tuple: Кортеж, содержащий список словарей, представляющих признаки, и список списков целевых значений.

    Пример:
    ```
    batch_size = 32
    batches_generator = generate_batches(check_data, batch_size=batch_size)
    for features, targets in batches_generator:
        # Обработка пакета
    ```
    """

    # Создаем словарь для хранения частот quantity для каждого продукта
    quantity_counts = {}

    # Считаем частоты
    for _, row in check_data.iterrows():
        product_id = row['product_id']
        quantity = int(row['quantity'])

        if product_id not in quantity_counts:
            quantity_counts[product_id] = {}

        if quantity not in quantity_counts[product_id]:
            quantity_counts[product_id][quantity] = 0

        quantity_counts[product_id][quantity] += 1

    # Для каждого продукта выбираем 5 самых частых quantity
    most_common_quantities = {}
    N = 5

    for product_id, quantities in quantity_counts.items():
        most_common_quantities[product_id] = [
            quantity for quantity, count in sorted(quantities.items(), key=lambda x: x[1], reverse=True)[:N]
        ]

    # Генератор пар фича-таргет и их деление на батчи
    while True:
        data = data.sample(frac=1).reset_index(drop=True)  # Перемешиваем данные
        for i in range(0, len(data), batch_size):
            batch = data.iloc[i:i + batch_size]

            features = []
            targets = []

            for _, row in batch.iterrows():
                product_id = row['product_id']
                quantity = int(row['quantity'])

                # Используем one-hot encoding для таргетов
                target = [1 if quantity in most_common_quantities[product_id] else 0 for quantity in range(1, N+1)]

                features.append({
                    'product_id': product_id,
                    'quantity': quantity
                })
                targets.append(target)

            yield features, targets

batch_size = 32
batches_generator = generate_batches(check_data, batch_size=batch_size)

## Рекомендации по выбранной архитектуре

In [ ]:
# Считаем размер словаря для модели
unique_product_ids = check_data['product_id'].unique()

vocab_size = len(unique_product_ids)

print(f"Размер словаря (vocab_size): {vocab_size}")

In [ ]:
# Определение модели
class Recommend(nn.Module):
    def __init__(self, vocab_size: int, hidden_size: int):
        super(Recommend, self).__init__()
        self.hidden = nn.Linear(vocab_size, hidden_size)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(hidden_size, vocab_size)

    def forward(self, x: torch.Tensor):
        out = self.hidden(x)
        out = self.relu(out)
        out = self.linear(out)
        return out

# Параметры модели и обучения
vocab_size = 36549
hidden_size = 100
learning_rate = 0.001

# Инициализация модели, функции потерь и оптимизатора
model = Recommend(vocab_size, hidden_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Обучение модели
num_epochs = 3
for epoch in range(num_epochs):
    for batch_features, batch_targets in batches_generator:
        features_tensor = torch.Tensor([item['product_id'] for item in batch_features])
        targets_tensor = torch.Tensor(batch_targets)

        # Прямой проход
        outputs = model(features_tensor)

        # Рассчет функции потерь
        loss = criterion(outputs, targets_tensor.argmax(dim=1))

        # Обратный проход и оптимизация
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

## Рекомендации по Content-Based Filtering

In [ ]:
# Выбор признаков для Content-Based Filtering
features = ['product_sub_category_id', 'product_category_id', 'brand_id']

# Создание модели Content-Based Filtering
model_cb = NearestNeighbors(metric='cosine', algorithm='brute')

# LOOCV - Leave One Out Cross-Validation
loo = LeaveOneOut()

recall_scores = []

for train_index, test_index in loo.split(check_data):
    # Разделяем данных на train и test для текущей итерации LOOCV
    train_data, test_data = check_data.iloc[train_index], check_data.iloc[test_index]

    # Обучаем модель Content-Based Filtering
    model_cb.fit(train_data[features])

    # Получаем рекомендации для текущего пользователя в тестовой выборке
    user_profile = test_data[features]
    distances, indices = model_cb.kneighbors([user_profile], n_neighbors=5)

    # Предсказание на основе ближайших соседей
    predicted_indices = indices.flatten()

    # Реальные продукты, которые пользователь взаимодействовал в тестовой выборке
    true_indices = test_data['product_id']

    # Оценка Recall@K
    recall_at_k = recall_score(true_indices, predicted_indices[:5], average='micro')
    recall_scores.append(recall_at_k)

# Средний Recall@K по всем итерациям LOOCV
average_recall = sum(recall_scores) / len(recall_scores)

print(f'Average Recall@{5}: {average_recall}')

## Сравнение качества модели Content-Based Filtering с двумя наивными моделями: случайные рекомендации и рекомендации наиболее популярных айтемов.

In [ ]:
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import recall_score

# Content-Based Filtering
model_cb = NearestNeighbors(metric='cosine', algorithm='brute')
loo = LeaveOneOut()

cb_recall_scores = []

for train_index, test_index in loo.split(grouped_df):
    train_data, test_data = grouped_df.iloc[train_index], grouped_df.iloc[test_index]
    model_cb.fit(train_data[features])

    user_profile = test_data[features]
    distances, indices = model_cb.kneighbors([user_profile], n_neighbors=5)
    predicted_indices = indices.flatten()
    true_indices = test_data['product_id']
    recall_at_k = recall_score(true_indices, predicted_indices[:5], average='micro')
    cb_recall_scores.append(recall_at_k)

random_recall_scores = []

for _ in range(len(grouped_df)):
    # Генерируем случайные рекомендации
    random_recommendations = np.random.choice(grouped_df['product_id'], size=3, replace=False)
    true_indices = grouped_df.iloc[_]['product_id']
    recall_at_k = recall_score([true_indices], random_recommendations, average='micro')
    random_recall_scores.append(recall_at_k)

popular_items_recall_scores = []

most_popular_items = grouped_df['product_id'].value_counts().index[:5]

for _ in range(len(grouped_df)):
    true_indices = grouped_df.iloc[_]['product_id']
    recall_at_k = recall_score([true_indices], most_popular_items, average='micro')
    popular_items_recall_scores.append(recall_at_k)

# Средний Recall@K по всем итерациям для каждой модели
average_cb_recall = mean(cb_recall_scores)
average_random_recall = mean(random_recall_scores)
average_popular_items_recall = mean(popular_items_recall_scores)

print(f'Average Recall@K for Content-Based Filtering: {average_cb_recall}')
print(f'Average Recall@K for Random Recommendations: {average_random_recall}')
print(f'Average Recall@K for Most Popular Items Recommendations: {average_popular_items_recall}')